# スクレイピング
Webからプログラムで入手するようには設計されていないデータを取得する行為

- requests: HTTPリクエストを行うライブラリ
- BeautifulSoup: 簡単なスクレイピングに便利
  - lxml: BeautifulSoupのデフォルトパーサ、高速に動作
- Scrapy: 大規模なデータスクレイピングに便利

## BeautifulSoup

### 設定

In [1]:
from bs4 import BeautifulSoup
import requests

In [2]:
URL = 'http://en.wikipedia.org/wiki/List_of_Nobel_laureates'
# httpheaderに「User-Agent」属性を追加しないと、 Wikipediaはリクエストを拒否する
HEADERS = {'User-Agent': 'Mozilla/5.0'}

### ページ取得

In [3]:
!pip install lxml

In [4]:
def get_Nobel_soup():
    """ ノーベル賞ページの解析したタグツリーを返す """
    response = requests.get(URL, headers=HEADERS)
    return BeautifulSoup(response.content, "lxml")

In [5]:
soup = get_Nobel_soup()

### テーブル解析

In [6]:
# 実行結果の表示領域が大きいのでスキップ中
# soup.find('table', {'class':'wikitable sortable'}) # 第1引数: タグ名, 第2引数: 各種属性のディクショナリ

In [7]:
soup.find('table', {'class':'sortable wikitable'}) # クラス順番が異なる場合は空を返す → select 使え（スープの作成時に lxml パーサを指定すると使用可能）

In [8]:
# 実行結果の表示領域が大きいのでスキップ中
# soup.select('table.sortable.wikitable') # ← は複数返す、1つのみの場合は select_one

In [9]:
wikitable = soup.select_one('table.sortable.wikitable')
wikitable.select('th') == wikitable('th') # スープに対してタグを直接呼び出せる

True

In [10]:
def get_column_titles(table):
    """ テーブルヘッダからノーベル賞分野を取得する """
    cols = []
    for th in table.select_one('tr').select('th')[1:]: #  ヘッダーはスキップ
        link = th.select_one('a')
        
        # 分野名と Wikipedia リンクを格納する
        if link:
            cols.append({'name': link.text, 'href': link.attrs['href']})
        else:
            cols.append({'name': th.text, 'href': None})

    return cols

In [11]:
wikitable = soup.select_one('table.sortable.wikitable')
get_column_titles(wikitable)

[{'name': 'Physics', 'href': '/wiki/List_of_Nobel_laureates_in_Physics'},
 {'name': 'Chemistry', 'href': '/wiki/List_of_Nobel_laureates_in_Chemistry'},
 {'name': 'Physiologyor Medicine',
  'href': '/wiki/List_of_Nobel_laureates_in_Physiology_or_Medicine'},
 {'name': 'Literature', 'href': '/wiki/List_of_Nobel_laureates_in_Literature'},
 {'name': 'Peace', 'href': '/wiki/List_of_Nobel_Peace_Prize_laureates'},
 {'name': 'Economics', 'href': '/wiki/List_of_Nobel_laureates_in_Economics'}]

### セル取得

In [12]:
def get_Nobel_winners(table):
    cols = get_column_titles(table)
    winners = []
    for row in table.select('tr')[1:-1]:
        year = int(row.select_one('td').text[:4]) # 2000[1]のような注釈付き対策
        for i, td in enumerate(row.select('td')[1:]):
            for winner in td.select('a'):
                href = winner.attrs['href']
                if not href.startswith('#endnote'):
                    winners.append({ 'year':year,'category':cols[i]['name'], 'name':winner.text, 'link':winner.attrs['href']})
    return winners

In [13]:
winners = get_Nobel_winners(wikitable)
winners[:5]

[{'year': 1901,
  'category': 'Physics',
  'name': 'Wilhelm Röntgen',
  'link': '/wiki/Wilhelm_R%C3%B6ntgen'},
 {'year': 1901,
  'category': 'Chemistry',
  'name': "Jacobus Henricus van 't Hoff",
  'link': '/wiki/Jacobus_Henricus_van_%27t_Hoff'},
 {'year': 1901,
  'category': 'Physiologyor Medicine',
  'name': 'Emil Adolf von Behring',
  'link': '/wiki/Emil_Adolf_von_Behring'},
 {'year': 1901,
  'category': 'Literature',
  'name': 'Sully Prudhomme',
  'link': '/wiki/Sully_Prudhomme'},
 {'year': 1901,
  'category': 'Peace',
  'name': 'Henry Dunant',
  'link': '/wiki/Henry_Dunant'}]

### キャッシュ

In [14]:
!pip install --upgrade requests-cache

Requirement already up-to-date: requests-cache in /opt/conda/lib/python3.7/site-packages (0.5.0)


In [15]:
import requests
import requests_cache
requests_cache.install_cache('nobel_pages', packend='sqlite', expire_after=7200) # nobel_pages という名前のキャッシュ

### 人物情報の取得

In [16]:
def get_winner_nationality(w):
    """ 受賞者の Wikipedia ページから人物情報データをスクレイピングする """
    url_str = 'http://en.wikipedia.org' + w['link']
    data = requests.get(url_str, headers=HEADERS)
    soup = BeautifulSoup(data.content, "lxml")
    person_data = {'name': w['name']}
    attr_rows = soup.select('table.infobox tr')
    for tr in attr_rows:
        try:
            attribute = tr.select_one('th').text
            if attribute == 'Nationality':
                person_data[attribute] = tr.select_one('td').text
        except AttributeError:
            pass
    return person_data

In [17]:
scraping_count = 10 # スクレイピングする回数(-1で全件)(国籍が存在しないなど取得に失敗しても1カウント)

wdata = []
for w in winners[:scraping_count]:
    wdata.append(get_winner_nationality(w))
    missing_nationality = []
for w in wdata:
    if not w.get('Nationality'):
        missing_nationality.append(w)

In [18]:
wdata

[{'name': 'Wilhelm Röntgen', 'Nationality': 'German[1]'},
 {'name': "Jacobus Henricus van 't Hoff", 'Nationality': 'Dutch'},
 {'name': 'Emil Adolf von Behring', 'Nationality': 'German'},
 {'name': 'Sully Prudhomme', 'Nationality': 'French'},
 {'name': 'Henry Dunant', 'Nationality': 'Swiss'},
 {'name': 'Frédéric Passy', 'Nationality': 'French'},
 {'name': 'Hendrik Lorentz', 'Nationality': 'Netherlands'},
 {'name': 'Pieter Zeeman', 'Nationality': 'Netherlands'},
 {'name': 'Hermann Emil Fischer', 'Nationality': 'Germany'},
 {'name': 'Ronald Ross', 'Nationality': 'British'}]

In [19]:
missing_nationality

[]

In [20]:
len(missing_nationality)

0

## Scrapy

In [21]:
!pip install scrapy

In [22]:
import os.path

if not os.path.exists('./nobel_winners'):
    print('try `scrapy startproject`')
    !scrapy startproject nobel_winners

フォルダ構成
<pre>
$ tree
.
├── nobel_winners
│   ├── __init__.py
│   ├── __pycache__
│   ├── items.py
│   ├── middlewares.py
│   ├── pipelines.py
│   ├── settings.py
│   └── spiders
│       ├── __init__.py
│       └── __pycache__
└── scrapy.cfg

4 directories, 7 files
</pre>

- `scrapy shell` で対話的に操作できる
- スパイダー: スクレイピング方法を定義するクラス
  - `scrapy.Spider` クラスのサブクラスとして実装
  - `spiders` ディレクトリに入れられたすべてのスパイダーは `Scrapy` が自動的に検出する → `scrapy` コマンドから名前でアクセス可
- キャッシュを有効化させる場合は `${project}/settings.py` の `HTTPCACHE_ENABLED` を `True` に設定する
- パイプライン: アイテムをスクレイピングした後に続けて行う処理
  - 複数定義可
  - Scrapy が定義済みのテンプレートのパイプラインもある
